<a href="https://colab.research.google.com/github/aastha12/dphi-machine-learning-bootcamp/blob/main/Quiz_1_Dphi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
insurance_data = pd.read_csv("https://github.com/dphi-official/Datasets/blob/master/travel_insurance/Training_set_label.csv?raw=true")

In [ ]:
insurance_data.head()

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age,Claim
0,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,61,UNITED KINGDOM,19.8,11.88,NaN,29,0
1,EPX,Travel Agency,Online,Cancellation Plan,93,NEW ZEALAND,63.0,0.00,NaN,36,0
2,EPX,Travel Agency,Online,2 way Comprehensive Plan,22,UNITED STATES,22.0,0.00,NaN,25,0
3,C2B,Airlines,Online,Silver Plan,14,SINGAPORE,54.5,13.63,M,24,0
4,EPX,Travel Agency,Online,Cancellation Plan,90,VIET NAM,10.0,0.00,NaN,23,0


In [ ]:
insurance_data.isnull().sum()

Agency                      0
Agency Type                 0
Distribution Channel        0
Product Name                0
Duration                    0
Destination                 0
Net Sales                   0
Commision (in value)        0
Gender                  34361
Age                         0
Claim                       0
dtype: int64

In [ ]:
import plotly.express as px

px.histogram(insurance_data,x='Duration')

In [ ]:
insurance_data.describe(include='all')

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age,Claim
count,48260,48260,48260,48260,48260.000000,48260,48260.000000,48260.000000,13899,48260.000000,48260.000000
unique,16,2,2,26,NaN,141,NaN,NaN,2,NaN,NaN
top,EPX,Travel Agency,Online,Cancellation Plan,NaN,SINGAPORE,NaN,NaN,M,NaN,NaN
freq,26712,34917,47434,14202,NaN,10068,NaN,NaN,7137,NaN,NaN
mean,NaN,NaN,NaN,NaN,49.546602,NaN,40.747647,9.811735,NaN,39.927269,0.014671
std,NaN,NaN,NaN,NaN,104.828147,NaN,48.823217,19.695339,NaN,14.027329,0.120231
min,NaN,NaN,NaN,NaN,-2.000000,NaN,-389.000000,0.000000,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,9.000000,NaN,18.000000,0.000000,NaN,35.000000,0.000000
50%,NaN,NaN,NaN,NaN,22.000000,NaN,27.000000,0.000000,NaN,36.000000,0.000000
75%,NaN,NaN,NaN,NaN,53.000000,NaN,48.000000,11.630000,NaN,43.000000,0.000000


In [ ]:
insurance_data[insurance_data['Claim']==1]['Gender'].value_counts(normalize=True)

F    0.52795
M    0.47205
Name: Gender, dtype: float64

In [ ]:
insurance_data['Gender'].fillna('Not Specified',inplace=True)
insurance_data.drop(['Distribution Channel', 'Destination' , 'Agency Type'],axis=1,inplace=True)



In [ ]:
insurance_data.dtypes

Agency                   object
Product Name             object
Duration                  int64
Net Sales               float64
Commision (in value)    float64
Gender                   object
Age                       int64
Claim                     int64
dtype: object

In [ ]:
agency=pd.get_dummies(insurance_data['Agency'],prefix='Agency')
gender=pd.get_dummies(insurance_data['Gender'],prefix='Gender')
product=pd.get_dummies(insurance_data['Product Name'],prefix='Product Name')

In [ ]:
final = pd.concat([insurance_data,agency,gender,product],axis=1)
final.drop(['Agency','Gender','Product Name'],axis=1,inplace=True)

In [ ]:
final.head(2)

,Duration,Net Sales,Commision (in value),Age,Claim,Agency_ADM,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,Agency_CWT,Agency_EPX,Agency_JWT,Agency_JZI,Agency_KML,Agency_LWC,Agency_RAB,Agency_SSI,Agency_TST,Agency_TTW,Gender_F,Gender_M,Gender_Not Specified,Product Name_1 way Comprehensive Plan,Product Name_2 way Comprehensive Plan,Product Name_24 Protect,Product Name_Annual Gold Plan,Product Name_Annual Silver Plan,Product Name_Annual Travel Protect Gold,Product Name_Annual Travel Protect Platinum,Product Name_Annual Travel Protect Silver,Product Name_Basic Plan,Product Name_Bronze Plan,Product Name_Cancellation Plan,Product Name_Child Comprehensive Plan,Product Name_Comprehensive Plan,Product Name_Gold Plan,Product Name_Individual Comprehensive Plan,Product Name_Premier Plan,Product Name_Rental Vehicle Excess Insurance,Product Name_Silver Plan,Product Name_Single Trip Travel Protect Gold,Product Name_Single Trip Travel Protect Platinum,Product Name_Single Trip Travel Protect Silver,Product Name_Spouse or Parents Comprehensive Plan,Product Name_Ticket Protector,Product Name_Travel Cruise Protect,Product Name_Travel Cruise Protect Family,Product Name_Value Plan
0,61,19.8,11.88,29,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,93,63.0,0.00,36,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Splitting into dependent and independent features

X=final.copy()
X.drop(['Claim'],axis=1,inplace=True)
y=final['Claim']

In [ ]:
#Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=101)

print("Shape of training features:",X_train.shape)
print("Shape of training labels:",y_train.shape)
print("Shape of testing features:",X_test.shape)
print("Shape of testing labels:",y_test.shape)

Shape of training features: (33782, 49)
Shape of training labels: (33782,)
Shape of testing features: (14478, 49)
Shape of testing labels: (14478,)


In [ ]:
y_train.value_counts(normalize=True)

0    0.984814
1    0.015186
Name: Claim, dtype: float64

In [ ]:
# Handling imbalanced classes

!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
# transform the dataset
oversample = SMOTE(sampling_strategy=1.0,random_state=25)

In [ ]:
X_smote,y_smote=oversample.fit_resample(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



In [ ]:
y_smote[y_smote==0].shape,y_smote[y_smote==1].shape

((33269,), (33269,))

In [ ]:
y_smote.shape

(66538,)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

LR=LogisticRegression(max_iter=2000)

LR.fit(X_smote,y_smote)
training_predictions=LR.predict(X_smote)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_smote,training_predictions))

              precision    recall  f1-score   support

           0       0.75      0.82      0.78     33269
           1       0.80      0.72      0.76     33269

    accuracy                           0.77     66538
   macro avg       0.77      0.77      0.77     66538
weighted avg       0.77      0.77      0.77     66538



In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_smote,training_predictions))

0.7578864103781047


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_smote,training_predictions))

0.7699960924584448


In [ ]:
test_predictions=LR.predict(X_test)
print(classification_report(y_test,test_predictions))

              precision    recall  f1-score   support

           0       0.99      0.82      0.90     14283
           1       0.05      0.66      0.09       195

    accuracy                           0.82     14478
   macro avg       0.52      0.74      0.49     14478
weighted avg       0.98      0.82      0.89     14478



In [ ]:
f1_score(y_test,test_predictions)*100

8.895066018068103

In [ ]:
accuracy_score(y_test,test_predictions)

0.8188976377952756

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT=DecisionTreeClassifier(random_state=1)

DT.fit(X_smote,y_smote)
training_predictions=DT.predict(X_smote)

In [ ]:
print(classification_report(y_smote,training_predictions))
print(f1_score(y_smote,training_predictions)*100)
print(accuracy_score(y_smote,training_predictions)*100)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     33269
           1       1.00      1.00      1.00     33269

    accuracy                           1.00     66538
   macro avg       1.00      1.00      1.00     66538
weighted avg       1.00      1.00      1.00     66538

99.8675855427488
99.86774474736241


In [ ]:
test_predictions=DT.predict(X_test)
print(classification_report(y_test,test_predictions))
print(f1_score(y_test,test_predictions)*100)
print(accuracy_score(y_test,test_predictions)*100)

              precision    recall  f1-score   support

           0       0.99      0.98      0.98     14283
           1       0.05      0.10      0.07       195

    accuracy                           0.96     14478
   macro avg       0.52      0.54      0.52     14478
weighted avg       0.97      0.96      0.97     14478

6.70194003527337
96.34618041165906


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF=RandomForestClassifier(random_state=1)

RF.fit(X_smote,y_smote)
training_predictions=RF.predict(X_smote)

In [ ]:
print(classification_report(y_smote,training_predictions))
print(f1_score(y_smote,training_predictions)*100)
print(accuracy_score(y_smote,training_predictions)*100)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     33269
           1       1.00      1.00      1.00     33269

    accuracy                           1.00     66538
   macro avg       1.00      1.00      1.00     66538
weighted avg       1.00      1.00      1.00     66538

99.86766121270453
99.86774474736241


In [ ]:
test_predictions=RF.predict(X_test)
print(classification_report(y_test,test_predictions))
print(f1_score(y_test,test_predictions)*100)
print(accuracy_score(y_test,test_predictions)*100)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     14283
           1       0.06      0.07      0.07       195

    accuracy                           0.97     14478
   macro avg       0.53      0.53      0.53     14478
weighted avg       0.97      0.97      0.97     14478

6.7469879518072275
97.32697886448405


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_RF = {'bootstrap': [True],
'max_depth': [10, 20],
'min_samples_leaf': [3, 4],
'min_samples_split': [4, 6],
'n_estimators': [100, 200]
}


param_grid_DT = {'max_leaf_nodes': [5, 10, 15, 20],
'min_samples_split': [4, 6, 8, 10]}


grid_search_1 = GridSearchCV(RF,param_grid = param_grid_RF,cv = 3,n_jobs = -1, verbose = 2)
grid_search_2 = GridSearchCV(DT,param_grid = param_grid_DT,cv = 3,n_jobs = -1, verbose = 2)

In [ ]:
tuned_RF=grid_search_1.fit(X_smote,y_smote)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  4.0min finished


In [ ]:
tuned_RF.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
tuned_DT=grid_search_2.fit(X_smote,y_smote)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    7.5s finished


In [ ]:
tuned_DT.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=20,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [ ]:
f1_score(y_test,tuned_DT.predict(X_test))*100

11.510791366906476

In [ ]:
f1_score(y_test,tuned_RF.predict(X_test))*100

8.378870673952642

### RFE

In [ ]:
from sklearn.feature_selection import RFE

RF=RandomForestClassifier(random_state=1)

selector=RFE(RF,step=1)

selector.fit(X_smote,y_smote)

RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=1, verbose=0,
                                     warm_start=False),
    n_features_to_select=None, step=1, verbose=0)

In [ ]:
len(selector.support_[selector.support_== True])

24

In [ ]:
X_train.columns

Index(['Duration', 'Net Sales', 'Commision (in value)', 'Age', 'Agency_ADM',
       'Agency_ART', 'Agency_C2B', 'Agency_CBH', 'Agency_CCR', 'Agency_CSR',
       'Agency_CWT', 'Agency_EPX', 'Agency_JWT', 'Agency_JZI', 'Agency_KML',
       'Agency_LWC', 'Agency_RAB', 'Agency_SSI', 'Agency_TST', 'Agency_TTW',
       'Gender_F', 'Gender_M', 'Gender_Not Specified',
       'Product Name_1 way Comprehensive Plan',
       'Product Name_2 way Comprehensive Plan', 'Product Name_24 Protect',
       'Product Name_Annual Gold Plan', 'Product Name_Annual Silver Plan',
       'Product Name_Annual Travel Protect Gold',
       'Product Name_Annual Travel Protect Platinum',
       'Product Name_Annual Travel Protect Silver', 'Product Name_Basic Plan',
       'Product Name_Bronze Plan', 'Product Name_Cancellation Plan',
       'Product Name_Child Comprehensive Plan',
       'Product Name_Comprehensive Plan', 'Product Name_Gold Plan',
       'Product Name_Individual Comprehensive Plan',
       'Product Na

In [ ]:
selector.ranking_

array([ 1,  1,  1,  1, 22, 13,  1, 24, 15, 23,  1,  1,  5,  1,  8,  1, 16,
        2,  7, 14,  1,  1,  1,  1,  1, 20, 10,  1,  3, 21, 11,  1,  1,  1,
       25,  6,  1, 18, 17,  1,  1,  1,  1,  9, 19,  4, 12, 26,  1])

In [ ]:
f1_score(y_test,selector.predict(X_test))*100

6.403940886699506

In [ ]:
accuracy_score(y_test,selector.predict(X_test))*100

97.3753280839895